In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
print(sys.path)

['/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/michal/venv/ML-3.8/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/michal/venv/ML-3.8/lib/python3.8/site-packages/IPython/extensions', '/home/michal/.ipython']


### Setup

#### Validate lab package version installation

In [4]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

TFX version: 0.25.0
KFP version: 1.0.4


## Setup local path to data, train, test folders 

In [6]:
import os
from pathlib import Path

notebook_path=os.getcwd()
local_data_dirpath = os.path.join(notebook_path, 'data')

local_train_dirpath = os.path.join(local_data_dirpath, "train")
local_train_filepath = os.path.join(local_train_dirpath, "train.csv")
local_test_dirpath = os.path.join(local_data_dirpath, "test")
local_test_filepath = os.path.join(local_test_dirpath, "test.csv")


In [28]:
!ls '{notebook_path}/pipeline/tests/resources/20210425_225924'

CsvExampleGen  ExampleValidator  Pusher     StatisticsGen  Trainer    Tuner
Evaluator      InfraValidator	 SchemaGen  tfx_metadata   Transform


In [29]:
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

import os
import json

import ml_metadata
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma


from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

from tfx.orchestration import metadata
from tfx.types import standard_artifacts

from tensorflow.python.lib.io import file_io


#connection_config = metadata_store_pb2.ConnectionConfig()
#connection_config.fake_database.SetInParent() # Sets an empty fake database proto.

# path to metadata.db
#METADATA_STORE = '/home/michal/artifact-store/tfx-titanic-training/20210416_133756/tfx_metadata/tfx-titanic-training/metadata.db'

METADATA_STORE = f'{notebook_path}/pipeline/tests/resources/20210425_225924/tfx_metadata/tfx-titanic-training/metadata.db'

metadata_connection_config=metadata.sqlite_metadata_connection_config(METADATA_STORE)

store = metadata_store.MetadataStore(metadata_connection_config)

Set enviorment variables

In [9]:
%cd {notebook_path}/pipeline

/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline/pipeline


List the registered artifact types.

In [30]:
for artifact_type in store.get_artifact_types():
    print(artifact_type.name)

Examples
Schema
ExampleStatistics
TransformGraph
TransformCache
ExampleAnomalies
HyperParameters
Model
ModelRun
ModelEvaluation
ModelBlessing
InfraBlessing
PushedModel


Display the registered execution types.

In [32]:
for execution_type in store.get_execution_types():
    print(execution_type.name)

tfx.components.example_gen.csv_example_gen.component.CsvExampleGen
tfx.components.common_nodes.importer_node.ImporterNode
tfx.components.common_nodes.resolver_node.ResolverNode
tfx.components.statistics_gen.component.StatisticsGen
tfx.components.transform.component.Transform
tfx.components.example_validator.component.ExampleValidator
tfx.components.schema_gen.component.SchemaGen
tfx.components.tuner.component.Tuner
tfx.components.trainer.component.Trainer
tfx.components.evaluator.component.Evaluator
tfx.components.infra_validator.component.InfraValidator
tfx.components.pusher.component.Pusher


List the registered context types.

In [33]:
for context_type in store.get_context_types():
    print(context_type.name)

pipeline
run
component_run


## Visualizing TFX artifacts

### Retrieve data analysis and validation artifacts


In [34]:
with metadata.Metadata(metadata_connection_config) as store:
    schema_artifacts = store.get_artifacts_by_type(standard_artifacts.Schema.TYPE_NAME)    
    stats_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleStatistics.TYPE_NAME)
    anomalies_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleAnomalies.TYPE_NAME)

In [35]:
schema_file = os.path.join(schema_artifacts[-1].uri, 'schema.pbtxt')
print("Generated schame file:{}".format(schema_file))

stats_path = stats_artifacts[-1].uri
train_stats_file = os.path.join(stats_path, 'train', 'stats_tfrecord')
eval_stats_file = os.path.join(stats_path, 'eval', 'stats_tfrecord')
print("Train stats file:{}, Eval stats file:{}".format(
    train_stats_file, eval_stats_file))

anomalies_path = anomalies_artifacts[-1].uri
train_anomalies_file = os.path.join(anomalies_path, 'train', 'anomalies.pbtxt')
eval_anomalies_file = os.path.join(anomalies_path, 'eval', 'anomalies.pbtxt')

print("Train anomalies file:{}, Eval anomalies file:{}".format(
    train_anomalies_file, eval_anomalies_file))

Generated schame file:/home/michal/artifact-store/tfx-titanic-training/20210425_225924/SchemaGen/schema/7/schema.pbtxt
Train stats file:/home/michal/artifact-store/tfx-titanic-training/20210425_225924/StatisticsGen/statistics/4/train/stats_tfrecord, Eval stats file:/home/michal/artifact-store/tfx-titanic-training/20210425_225924/StatisticsGen/statistics/4/eval/stats_tfrecord
Train anomalies file:/home/michal/artifact-store/tfx-titanic-training/20210425_225924/ExampleValidator/anomalies/6/train/anomalies.pbtxt, Eval anomalies file:/home/michal/artifact-store/tfx-titanic-training/20210425_225924/ExampleValidator/anomalies/6/eval/anomalies.pbtxt


### Visualize schema

In [36]:
schema = tfdv.load_schema_text(schema_file)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Embarked',STRING,required,,'Embarked'
'Name',BYTES,required,single,-
'Sex',STRING,required,single,'Sex'
'Ticket',BYTES,required,single,-
'Cabin',BYTES,required,,-
'Age',FLOAT,required,,-
'Fare',FLOAT,required,single,-
'Parch',INT,required,single,-
'PassengerId',INT,required,single,-


/home/michal/venv/ML-3.8/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Embarked',"'C', 'Q', 'S'"
'Sex',"'female', 'male'"


Visualize statistics


In [37]:
train_stats = tfdv.load_statistics(train_stats_file)
eval_stats = tfdv.load_statistics(eval_stats_file)
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

Visualize anomalies

In [38]:
train_anomalies = tfdv.load_anomalies_text(train_anomalies_file)
tfdv.display_anomalies(train_anomalies)

In [39]:
eval_anomalies = tfdv.load_anomalies_text(eval_anomalies_file)
tfdv.display_anomalies(eval_anomalies)

Retrieve model artifacts

In [40]:
with metadata.Metadata(metadata_connection_config) as store:
    model_eval_artifacts = store.get_artifacts_by_type(standard_artifacts.ModelEvaluation.TYPE_NAME)
    hyperparam_artifacts = store.get_artifacts_by_type(standard_artifacts.HyperParameters.TYPE_NAME)
    
model_eval_path = model_eval_artifacts[-1].uri
print("Generated model evaluation result:{}".format(model_eval_path))
best_hparams_path = os.path.join(hyperparam_artifacts[-1].uri, 'best_hyperparameters.txt')
print("Generated model best hyperparameters result:{}".format(best_hparams_path))

Generated model evaluation result:/home/michal/artifact-store/tfx-titanic-training/20210425_225924/Evaluator/evaluation/10
Generated model best hyperparameters result:/home/michal/artifact-store/tfx-titanic-training/20210425_225924/Tuner/best_hyperparameters/8/best_hyperparameters.txt


### Return best hyperparameters

In [41]:
# Latest pipeline run Tuner search space.
json.loads(file_io.read_file_to_string(best_hparams_path))['space']

[{'class_name': 'Float',
  'config': {'name': 'learning_rate',
   'default': 0.0009167702421017742,
   'conditions': [],
   'min_value': 0.0001,
   'max_value': 0.01,
   'step': None,
   'sampling': 'log'}},
 {'class_name': 'Int',
  'config': {'name': 'n_layers',
   'default': 2,
   'conditions': [],
   'min_value': 1,
   'max_value': 2,
   'step': 1,
   'sampling': None}},
 {'class_name': 'Int',
  'config': {'name': 'n_units_1',
   'default': 72,
   'conditions': [{'class_name': 'Parent',
     'config': {'name': 'n_layers', 'values': [1]}}],
   'min_value': 8,
   'max_value': 128,
   'step': 8,
   'sampling': None}},
 {'class_name': 'Int',
  'config': {'name': 'n_units_1',
   'default': 128,
   'conditions': [{'class_name': 'Parent',
     'config': {'name': 'n_layers', 'values': [2]}}],
   'min_value': 8,
   'max_value': 128,
   'step': 8,
   'sampling': None}},
 {'class_name': 'Int',
  'config': {'name': 'n_units_2',
   'default': 80,
   'conditions': [{'class_name': 'Parent',
     '

In [42]:
# Latest pipeline run Tuner searched best_hyperparameters artifacts.
json.loads(file_io.read_file_to_string(best_hparams_path))['values']

{'learning_rate': 0.0014421563688159582, 'n_layers': 1, 'n_units_1': 32}

### Visualize model evaluations

In [46]:
eval_result = tfma.load_eval_result(model_eval_path)
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='Sex')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Sex:female', 'metric…